**IMPORTANT** 

- For requirements and initial setup go to https://github.com/OliveiraEdu/OpenScience/Readme.md;
- To execute the notebook run all cells.

# Notebook 2 - Open Science Platform - User Account Creation

## Activities

1 - Deploys a smart contract into the Iroha 1 blockchain for account creation;

2 - Creates a user account into the blockchain;

3 - Queries the blockchain to confirm the succesful creation of the account;

4 - Deploys a smart contract into the Iroha 1 blockchain for details (attributes) setting;

5 - Sets details for the user account;

6 - Queries the blockchain again to confirm the succesfull details setting.

User accounts and their respective attributes are randomly picked, every account succesfully created is dumped to `datasets/accounts.json` for later use and evidence collection.  

In [10]:
from name_generator import left, right
from utilities import *
from dump_to_json import *
from iroha_helper import *
from new_helper import extract_account_metadata_cid_from_data, process_raw_data, load_variable
from loguru import logger

In [11]:
DOMAIN = "test"

1 - Loads the smart contract hash for account creation into the Iroha 1 blockchain.

In [12]:
# Loading the variable from JSON
create_account_contract_hash = load_variable("Create Account Hash")
logger.info(create_account_contract_hash)


2025-02-08 22:48:29,894 - INFO - Variable 'Create Account Hash' successfully loaded from temp/Create Account Hash.json
2025-02-08 22:48:29.896 | INFO     | __main__:<module>:3 - b'a76bb4a33334ef018daf965042478cfc41229121be8e4d461a3dc9f9c709c2ce'


2 - Creates a user account into the blockchain.

In [13]:
user_private_key = IrohaCrypto.private_key()
user_public_key = IrohaCrypto.derive_public_key(user_private_key).decode("utf-8")

# - Random generated synthetic user data 
user_account_left = random.choice(left)
user_account_right = random.choice(right)
user_account_short_id = f"{user_account_left}_{user_account_right}"
user_account_full_name = ((f"{user_account_left}").capitalize())+" "+((f"{user_account_right}").capitalize())
user_account_email = f"{user_account_left}_{user_account_right}"+"@email.com"
user_account_institution = print_random_from_second_column("datasets/world-universities.csv")
user_account_orcid = generate_orcid()
user_role = set_random_role()
logger.info(f"User Role: {user_role}")

user_account = UserAccount(
    account_id=f"{user_account_short_id}@{DOMAIN}",
    full_name=user_account_full_name,
    email=user_account_email,
    institution=user_account_institution,
    orcid=user_account_orcid,
    role=user_role,
    public_key=user_public_key,
)
logger.info(f"Creating account with name: {user_account_short_id}")

# - Create account
create_account_contract_address = integration_helpers.get_engine_receipts_address(create_account_contract_hash)
create_user_account(create_account_contract_address, user_account_short_id, DOMAIN, user_public_key, user_account)

2025-02-08 22:48:29.991 | INFO     | __main__:<module>:13 - User Role: publisher
2025-02-08 22:48:29.993 | INFO     | __main__:<module>:24 - Creating account with name: amazing_joliot
2025-02-08 22:48:29.994 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_address"
2025-02-08 22:48:30.013 | INFO     | integration_helpers:get_engine_receipts_address:96 - Retrieved contract address for transaction hash: b'a76bb4a33334ef018daf965042478cfc41229121be8e4d461a3dc9f9c709c2ce'
2025-02-08 22:48:30.015 | DEBUG    | integration_helpers:get_engine_receipts_address:97 - Contract address: 93F622673AA712B7298FDD2212E3228D8D9DD8BF
2025-02-08 22:48:30.016 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_address"
2025-02-08 22:48:30.017 | DEBUG    | integration_helpers:tracer:35 - 	Entering "create_user_account"
2025-02-08 22:48:30,033 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-02-08 22:48:30,034 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)

2025-02-08 22:48:30,562 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-02-08 22:48:30,571 - INFO - ('COMMITTED', 5, 0)
2025-02-08 22:48:30.576 | INFO     | dump_to_json:dump_to_json_ld:82 - Appended new entry to file 'datasets/accounts.json', current total entries: 12
2025-02-08 22:48:30.578 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "create_user_account"


b'8918f895abe62603252f91b757fb300ef9459cc398dd86a1581269a9632f63f6'

3 - Queries the blockchain to confirm the succesful creation of the account.

In [14]:
# - Get account info
get_account_hash = get_account(create_account_contract_address, user_account_short_id, DOMAIN)
address = integration_helpers.get_engine_receipts_result(get_account_hash)

2025-02-08 22:48:30.601 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_account"
2025-02-08 22:48:30,603 - WARNING - create_account_contract_address: 93F622673AA712B7298FDD2212E3228D8D9DD8BF
2025-02-08 22:48:30,620 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-02-08 22:48:30,622 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-02-08 22:48:33,593 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-02-08 22:48:33,601 - INFO - ('COMMITTED', 5, 0)
2025-02-08 22:48:33,603 - WARNING - Account Data: {'address': '93F622673AA712B7298FDD2212E3228D8D9DD8BF', 'hex_hash': '74fc2608e663158d39e8f4ee22afbc8abde35797f3dfe97c0983a1c006b09faa', 'timestamp': 1739054913603}
2025-02-08 22:48:33.604 | DEBUG    | integration_helpers:tracer:35 - 	Entering "append_to_json_file"
2025-02-08 22:48:33,607 - INFO - Appended data under 'amazing_joliot@test' in logs/account_data.json
2025-02-08 22:48:33.608 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "append_to_json_file"
2025-02-08 22:

4 - Loads the smart contract for details (attributes) setting into the Iroha 1 blockchain .

In [15]:
# Loading the variable from JSON
account_detail_contract_hash = load_variable("Account Detail Contract Hash")
logger.info(account_detail_contract_hash)

2025-02-08 22:48:33,652 - INFO - Variable 'Account Detail Contract Hash' successfully loaded from temp/Account Detail Contract Hash.json
2025-02-08 22:48:33.654 | INFO     | __main__:<module>:3 - b'86452d50ed5d8cdb132dc84c548608c8454a8a993a6deab3384e17e1aee55ec0'


5 - Sets details for the user account:

- Full Name
- Email
- Institution
- ORCID
- Public key
- Role

In [16]:
address = integration_helpers.get_engine_receipts_address(account_detail_contract_hash)

process_account(address, f"{user_account_short_id}@{DOMAIN}")

2025-02-08 22:48:33.712 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_address"
2025-02-08 22:48:33.733 | INFO     | integration_helpers:get_engine_receipts_address:96 - Retrieved contract address for transaction hash: b'86452d50ed5d8cdb132dc84c548608c8454a8a993a6deab3384e17e1aee55ec0'
2025-02-08 22:48:33.735 | DEBUG    | integration_helpers:get_engine_receipts_address:97 - Contract address: A518E3EE4D45C20EF834361093305D2500A46228
2025-02-08 22:48:33.736 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_address"
2025-02-08 22:48:33.737 | DEBUG    | integration_helpers:tracer:35 - 	Entering "process_account"
2025-02-08 22:48:33,739 - INFO - Processing account: amazing_joliot@test
2025-02-08 22:48:33,740 - INFO - User Account Metadata: {'@type': 'foaf:Person', 'foaf:name': 'Amazing Joliot', 'foaf:mbox': 'amazing_joliot@email.com', 'foaf:organization': {'@type': 'foaf:Organization', 'foaf:name': 'Technische Fachhochschule Wildau'}, 's

2025-02-08 22:48:36,621 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-02-08 22:48:36,630 - INFO - ('COMMITTED', 5, 0)
2025-02-08 22:48:36.631 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_result"
2025-02-08 22:48:36.653 | INFO     | integration_helpers:get_engine_receipts_result:139 - Result data: 000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000000000000200000000000000000000000000000000000000000000000000000000000000001
2025-02-08 22:48:36.654 | DEBUG    | integration_helpers:get_engine_receipts_result:140 - Result decoded string:                                                                                                
2025-02-08 22:48:36.655 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_result"
2025-02-08 22:48:36.657 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "set_account_detail"
2025-02-08 22:48:36.659 | DEBUG    | integration_helpers

6 - Queries the blockchain again to confirm the succesfull details setting.

In [17]:
# - Get account info
get_account_hash = get_account(create_account_contract_address, user_account_short_id, DOMAIN)
details = integration_helpers.get_engine_receipts_result(get_account_hash)

# Process the raw data
account_metadata_cid = process_raw_data(details)

2025-02-08 22:48:36.677 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_account"
2025-02-08 22:48:36,679 - WARNING - create_account_contract_address: 93F622673AA712B7298FDD2212E3228D8D9DD8BF
2025-02-08 22:48:36,695 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-02-08 22:48:36,697 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-02-08 22:48:39,653 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-02-08 22:48:39,661 - INFO - ('COMMITTED', 5, 0)
2025-02-08 22:48:39,664 - WARNING - Account Data: {'address': '93F622673AA712B7298FDD2212E3228D8D9DD8BF', 'hex_hash': '6db34b7c6d436070a47c1db1c960bb8fb0c3c6cff7d844edc0af393d931a73b8', 'timestamp': 1739054919664}
2025-02-08 22:48:39.665 | DEBUG    | integration_helpers:tracer:35 - 	Entering "append_to_json_file"
2025-02-08 22:48:39,668 - INFO - Appended data under 'amazing_joliot@test' in logs/account_data.json
2025-02-08 22:48:39.669 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "append_to_json_file"
2025-02-08 22:

7 - Retrieves User Metadata (JSON-LD formatted) from IPFS

In [18]:
user_metadata = download_json_from_ipfs(account_metadata_cid)
logger.info(f"User Metadata: {user_metadata}")

2025-02-08 22:48:39.726 | INFO     | __main__:<module>:2 - User Metadata: {'@type': 'foaf:Person', 'foaf:holdsAccount': {'schema:identifier': 'amazing_joliot@test', 'schema:publicKey': '7ba659433d45a246e2d1649b6ba8d5d88e1d9c0bfabe5560653df192a027efbc', 'schema:roleName': 'publisher'}, 'foaf:mbox': 'amazing_joliot@email.com', 'foaf:name': 'Amazing Joliot', 'foaf:organization': {'@type': 'foaf:Organization', 'foaf:name': 'Technische Fachhochschule Wildau'}, 'schema:identifier': {'@type': 'PropertyValue', 'propertyID': 'ORCID', 'value': '7073-6334-8506-X'}}
